In [1]:
# https://dplyr.tidyverse.org/reference/summarise.html

from datar.datasets import mtcars, starwars
from datar.all import *

In [2]:
mtcars >> summarise(mean=mean(f.disp), n=n())

,mean,n
0,230.721875,32


In [3]:
mtcars >> \
  group_by(f.cyl) >> \
  summarise(mean=mean(f.disp), n=n()) >> \
  display()

[2021-04-03 00:52:56][datar][   INFO] `summarise()` regrouping output by ['cyl']. You can override using the `.groups` argument.
[2021-04-03 00:52:56][datar][   INFO] # [DataFrameGroupBy] Groups: ['cyl'] (3)


,cyl,mean,n
0,4,105.136364,11
1,6,183.314286,7
2,8,353.100000,14


In [4]:
summarise.inform = False
mtcars >> \
  group_by(f.cyl) >> \
  summarise(mean=mean(f.disp), n=n()) >> \
  display()

[2021-04-03 00:52:56][datar][   INFO] # [DataFrameGroupBy] Groups: ['cyl'] (3)


,cyl,mean,n
0,4,105.136364,11
1,6,183.314286,7
2,8,353.100000,14


In [5]:
summarise.inform = True

mtcars >> \
   group_by(f.cyl) >> \
   summarise(qs=quantile(f.disp, c(0.25, 0.75)), prob=c(0.25, 0.75)) >> \
   display()

[2021-04-03 00:52:56][datar][   INFO] `summarise()` regrouping output by ['cyl']. You can override using the `.groups` argument.
[2021-04-03 00:52:56][datar][   INFO] # [DataFrameGroupBy] Groups: ['cyl'] (3)


,cyl,qs,prob
0,4,78.85,0.25
1,4,120.65,0.75
2,6,160.00,0.25
3,6,196.30,0.75
4,8,301.75,0.25
5,8,390.00,0.75


In [6]:
_ >> display()

,cyl,qs,prob
0,4,78.85,0.25
1,4,120.65,0.75
2,6,160.00,0.25
3,6,196.30,0.75
4,8,301.75,0.25
5,8,390.00,0.75


In [7]:
from pipda import register_func, Context

def my_quantile(x, probs):
  return tibble(x=quantile(x, probs), probs = probs)

my_quantile = register_func(None, context=Context.EVAL, func=my_quantile)

mtcars >> \
  group_by(f.cyl) >> \
  summarise(my_quantile(f.disp, c(0.25, 0.75))) >> \
  display()


[2021-04-03 00:52:56][datar][   INFO] `summarise()` regrouping output by ['cyl']. You can override using the `.groups` argument.
[2021-04-03 00:52:56][datar][   INFO] # [DataFrameGroupBy] Groups: ['cyl'] (3)


,cyl,x,probs
0,4,78.85,0.25
1,4,120.65,0.75
2,6,160.00,0.25
3,6,196.30,0.75
4,8,301.75,0.25
5,8,390.00,0.75


In [8]:
mtcars >> \
  group_by(f.cyl, f.vs) >> \
  summarise(cyl_n = n()) >> \
  group_vars()

[2021-04-03 00:52:56][datar][   INFO] `summarise()` regrouping output by ['cyl', 'vs']. You can override using the `.groups` argument.


['cyl', 'vs']

In [9]:
# Unlike dplyr's summarise, f.disp can be reused.
mtcars >> \
  group_by(f.cyl) >> \
  summarise(disp=mean(f.disp), sd=sd(f.disp)) >> \
  display()

[2021-04-03 00:52:56][datar][   INFO] `summarise()` regrouping output by ['cyl']. You can override using the `.groups` argument.
[2021-04-03 00:52:56][datar][   INFO] # [DataFrameGroupBy] Groups: ['cyl'] (3)


,cyl,disp,sd
0,4,105.136364,26.871594
1,6,183.314286,41.562460
2,8,353.100000,67.771324


In [10]:
var = "mass"
starwars >> summarise(avg = mean(f[var], na_rm = TRUE))

,avg
0,97.311864
